In [1]:
import pandas as pd
import numpy as np
import re

In [5]:
df_raw = pd.read_csv("2024 Faculty MISO Survey Results (1).csv", encoding="ISO-8859-1")

In [7]:
question_text_row = df_raw.iloc[0]   # Row 0: actual question descriptions
importid_row = df_raw.iloc[1]        # Row 1: structured ImportIds

In [9]:
def extract_importid(col_name):
    match = re.search(r'"ImportId":"(QID\d+_\d+|QID\d+)"', str(col_name))
    return match.group(1) if match else None

# Use the ImportId row to rename the columns
df_raw.columns = [extract_importid(val) for val in importid_row]

In [11]:
df_cleaned = df_raw.iloc[2:].copy()

In [13]:
df_cleaned = df_cleaned.apply(pd.to_numeric, errors='coerce')  # Non-numeric values become NaN
df_cleaned = df_cleaned.replace(-99, np.nan)                   # Replace error codes
threshold = len(df_cleaned) * 0.5
df_cleaned = df_cleaned.dropna(thresh=threshold, axis=1)       # Drop mostly-empty columns

In [15]:
demographic_prefixes = ['QID31', 'QID35', 'QID32_TEXT', 'QID60_7_TEXT', 'QID30', 'QID29']
columns_to_keep = [col for col in df_cleaned.columns if not any(prefix in str(col) for prefix in demographic_prefixes)]
df_filtered = df_cleaned[columns_to_keep]

In [27]:
df_filtered.to_csv("Kaveh_2024_Faculty_Cleaned.csv", index=False)

In [17]:
lowest_scores = df_filtered.mean().sort_values().head(5)

In [21]:
# Reload metadata rows for fresh mapping
df_meta = pd.read_csv("2024 Faculty MISO Survey Results (1).csv", encoding="ISO-8859-1")
question_text_row = df_meta.iloc[0]
importid_row = df_meta.iloc[1]

# Rebuild map: ImportId -> Question Text
importid_map = {}
for col in df_meta.columns:
    id_ = extract_importid(importid_row[col])
    if id_:
        importid_map[id_] = question_text_row[col]

# Match the cleaned columns to real descriptions
lowest_question_texts = {qid: importid_map.get(qid, "Unknown") for qid in lowest_scores.index}


In [23]:
print(lowest_scores)
print(lowest_question_texts)

QID17_1    0.226277
QID17_6    0.268116
QID17_4    0.270073
QID18_6    0.358209
QID18_1    0.362963
dtype: float64
{'QID17_1': 'Do you use the following tools for academic purposes? - Classroom electronic polling (e.g. clickers, Poll Everywhere)', 'QID17_6': 'Do you use the following tools for academic purposes? - Student electronic portfolios', 'QID17_4': 'Do you use the following tools for academic purposes? - Lecture capture', 'QID18_6': 'Do you personally own the following devices? - E-book reader (e.g. Kindle, NOOK)', 'QID18_1': 'Do you personally own the following devices? - Desktop computer'}


In [29]:
# Create a DataFrame combining the scores and their matching question texts
insight_df = pd.DataFrame({
    'Import ID': lowest_scores.index,
    'Avg. Score': lowest_scores.values,
    'Question Text': [lowest_question_texts[qid] for qid in lowest_scores.index]
})

# Display as a nice table
print(insight_df)

  Import ID  Avg. Score                                      Question Text
0   QID17_1    0.226277  Do you use the following tools for academic pu...
1   QID17_6    0.268116  Do you use the following tools for academic pu...
2   QID17_4    0.270073  Do you use the following tools for academic pu...
3   QID18_6    0.358209  Do you personally own the following devices? -...
4   QID18_1    0.362963  Do you personally own the following devices? -...


In [31]:
# Save insight as file too
insight_df.to_csv("K_2024_Faculty_Insight_Top5.csv", index=False)